#### setdefault나 defaultdict 모두 사용하기 적당하지 않은 경우도 있다.
- 디폴트 값을 만드는 계산 비용이 높거나 만드는 과정에서 setdefault를 사용하지 말라
- default 함수를 사용해야하지만 전달되는 함수 인자는 받지 않는다, 그래서 __missing__메서드로 정의하여 사용하라

In [17]:
#get함수 예시
# SNS프로필 사진을 관리하는 프로그램을 작성할 때 예시
#필요할 때 파일을 읽고 쓰기 위해 프로필 사진 경로와 열린 파일 핸들을 연관시켜주는 딕셔너리가 필요.
# 블록의 깊이가 깊어진다.
pictures = {}
path = 'profile_1234.png'

if (handle := pictures.get(path)) is None: #none값으로 true
    try:
        handle = open(path, 'a+b') # 이 구문이 실행
    except OSError:
        print(f'경로를 열 수 없습니다: {path}')
        raise
    else:
        pictures[path] = handle

handle.seek(0)
image_data = handle.read()
print(handle.seek(0)) #데이터의 바이트를 이동
print(image_data)

0
b''


- file : 파일 경로
- mode : 파일이 열리는 모드
- 'r' : 읽기 용으로 열림 (기본값)
- 'w' : 쓰기 위해 열기, 파일을 먼저 자른다.
- 'x' : 베타적 생성을 위해 열리고, 이미 존재하는 경우 실패
- 'a' : 쓰기를 위해 열려 있고, 파일의 끝에 추가하는 경우 추가합니다.
- 'b' : 2진 모드(바이너리 모드)
- 't' : 텍스트 모드 (기본값)
- '+' : 업데이트 (읽기 및 쓰기)를 위한 디스크 파일 열기
- 'U' : 유니버설 개행 모드 (사용되지 않음)

In [4]:
# setdefault 예시

try:
    handle = pictures.setdefault(path, open(path, 'a+b'))
except OSError:
    print(f'경로를 열 수 없습니다: {path}')
    raise
else:
    handle.seek(0)
    image_data = handle.read()

print(handle.seek(0)) #데이터의 바이트를 읽음
print(image_data)   

0
b''


##### setdefault의 단점
- 딕셔너리 경로가 있는지 여부와 관계없이 항상 호출
- 핸들링과정에서 새로운 파일이 생길 수 있음
- 예외 처리가 가능하지만 setdefault의 예외와 구분을 못할 수 있다. (43때 다시..)

In [18]:
# defaultdict 예시 
#의문점을 수정해봄
from collections import defaultdict

def open_picture(path):
    try:
        return open(path, 'a+b')
    except OSError:
        print(f'경로를 열 수 없습니다: {profile_path}')
        raise

# 오류가 나는 부분. 오류를 보고 싶으면 커멘트를 해제할것
pictures = defaultdict(open_picture)
handle = pictures[path]
handle.seek(0)
image_data = handle.read()

defaultdict(<function open_picture at 0x000001F1A6D5A040>, {})


TypeError: open_picture() missing 1 required positional argument: 'path'

- defaultdict가 호출하는 도우미 함수가 처리 중인 키를 알 수 없다는 뜻
- 파일 경로를 통해 open을 할 방법이 없다.

#### dict 타입의 하위 클래스를 만들고 __missing__ 특별 메서드를 구현하면 키가 없는 경우를 처리하는 로직을 커스텀화 할 수 있다.

In [20]:
class Pictures(dict):
    def __missing__(self, key):
        value = open_picture(key)
        self[key] = value
        return value

pictures = Pictures()
handle = pictures[path]
handle.seek(0)
image_data = handle.read()
print(image_data)

b''


In [7]:
x= {'x':1,'y':2, 'z' :3}

In [8]:
x.setdefault('a',4)
print(x)

{'x': 1, 'y': 2, 'z': 3, 'a': 4}


In [11]:
x= {'x':1,'y':2, 'z' :3}
x.defaultdict('x')
print(x)

AttributeError: 'dict' object has no attribute 'defaultdict'